**Import Libraries**

In [25]:
import pandas as pd
import numpy as np
from typing import List, Dict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import regex as re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download ('stopwords')
from nltk.corpus import stopwords 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Mount Drive**

In [26]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/dataset/IRproject

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/dataset/IRproject


**Read Dataset**

In [27]:
songs = pd.read_csv('songdata.csv')
songs

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \nLet the angels fly l...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \nMore power \nPower to...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \nis something i'll believe \nf...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \nam i frightened \nwhere can ...


**Select top 10000 songs**

In [28]:
songs2 = songs
songs = songs2[0:10000]

In [29]:
songs

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
9995,Justin Bieber,All Bad,/j/justin+bieber/all+bad_21071754.html,"[Verse:] \nIt's another, if it ain't one thin..."
9996,Justin Bieber,All I Want Is You,/j/justin+bieber/all+i+want+is+you_20985618.html,Sitting here all alone watching the snow fall ...
9997,Justin Bieber,As Long As You Love Me,/j/justin+bieber/as+long+as+you+love+me_210218...,"As long as you love me (love me, love me) \nA..."
9998,Justin Bieber,Baby Lady,/j/justin+bieber/baby+lady_20467491.html,Put me in a outfit from the gap like that. One...


**Preprocessing**

In [30]:
def preprocess(texts):
  stopword = stopwords.words('english')
  clean_text = ' '.join([word for word in texts.split()if word not in stopword]) 
  clean_text = re.sub('\n','',clean_text)
  clean_text = clean_text.lower()
  clean_text = re.sub(r'\[[^)]*\]','',clean_text)
  return clean_text


In [31]:
songs['Cleaned_text'] = songs['text'].apply(lambda text: preprocess(text))
songs

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,artist,song,link,text,Cleaned_text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd...","look face, wonderful face and means something ..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...","take easy me, please touch gently like summer ..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,i'll never know i go why i put lousy rotten sh...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,making somebody happy question give take you l...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,making somebody happy question give take you l...
...,...,...,...,...,...
9995,Justin Bieber,All Bad,/j/justin+bieber/all+bad_21071754.html,"[Verse:] \nIt's another, if it ain't one thin...","it's another, ain't one thing instigators, li..."
9996,Justin Bieber,All I Want Is You,/j/justin+bieber/all+i+want+is+you_20985618.html,Sitting here all alone watching the snow fall ...,sitting alone watching snow fall looking back ...
9997,Justin Bieber,As Long As You Love Me,/j/justin+bieber/as+long+as+you+love+me_210218...,"As long as you love me (love me, love me) \nA...","as long love (love me, love me) as long love (..."
9998,Justin Bieber,Baby Lady,/j/justin+bieber/baby+lady_20467491.html,Put me in a outfit from the gap like that. One...,put outfit gap like that. one matches with jac...


**Tf-idf**

In [32]:
tfidf = TfidfVectorizer()
lyrics_matrix = tfidf.fit_transform(songs['Cleaned_text'])
songs

,artist,song,link,text,Cleaned_text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd...","look face, wonderful face and means something ..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...","take easy me, please touch gently like summer ..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,i'll never know i go why i put lousy rotten sh...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,making somebody happy question give take you l...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,making somebody happy question give take you l...
...,...,...,...,...,...
9995,Justin Bieber,All Bad,/j/justin+bieber/all+bad_21071754.html,"[Verse:] \nIt's another, if it ain't one thin...","it's another, ain't one thing instigators, li..."
9996,Justin Bieber,All I Want Is You,/j/justin+bieber/all+i+want+is+you_20985618.html,Sitting here all alone watching the snow fall ...,sitting alone watching snow fall looking back ...
9997,Justin Bieber,As Long As You Love Me,/j/justin+bieber/as+long+as+you+love+me_210218...,"As long as you love me (love me, love me) \nA...","as long love (love me, love me) as long love (..."
9998,Justin Bieber,Baby Lady,/j/justin+bieber/baby+lady_20467491.html,Put me in a outfit from the gap like that. One...,put outfit gap like that. one matches with jac...


**Create BOW**

In [33]:
df1 = pd.DataFrame(lyrics_matrix.toarray(), columns=tfidf.get_feature_names())
bow_songs = songs
bow_songs = bow_songs.rename(columns={'artist': 'artist_data'})
bow_songs = bow_songs.rename(columns={'song': 'song_data'})
bow_songs = pd.concat([bow_songs, df1], axis=1)
bow_songs

,artist_data,song_data,link,text,Cleaned_text,00,000,009,00a,01,03,06,07,0h,0o,10,100,101,10174,102,103,105,10538,109,10th,10w40,11,110,115,117,11th,12,121,124,125,12th,13,13th,13x,14,...,zephyrus,zeppelin,zere,zero,zeroes,zeros,zeta,zeus,zi,ziegfeld,ziggy,zigzag,zing,zion,zip,ziplock,zipper,zits,zodiac,zom,zombie,zombies,zoms,zone,zones,zoney,zonin,zonked,zoo,zoom,zooming,zooms,zoos,zu,zugi,zukka,zulu,zuruck,zwei,zydeco
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd...","look face, wonderful face and means something ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...","take easy me, please touch gently like summer ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,i'll never know i go why i put lousy rotten sh...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,making somebody happy question give take you l...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,making somebody happy question give take you l...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Justin Bieber,All Bad,/j/justin+bieber/all+bad_21071754.html,"[Verse:] \nIt's another, if it ain't one thin...","it's another, ain't one thing instigators, li...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,Justin Bieber,All I Want Is You,/j/justin+bieber/all+i+want+is+you_20985618.html,Sitting here all alone watching the snow fall ...,sitting alone watching snow fall looking back ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

**Find Cosine Similarity**

In [34]:
cosine_similarities = cosine_similarity(lyrics_matrix)

In [44]:
similarities = {}
for i in range(len(cosine_similarities)):
    similarData = cosine_similarities[i].argsort()[:-100:-1] 
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similarData]

In [46]:
import pickle
# with open("stored_recommendation", "wb") as f:
pickle.dump(similarities, open("final_stored_recommendation", 'wb'))

**Recomendation**

In [39]:
def print_data(song, recommend_song):
    print('Recommended songs for '+song+' are: ')
    for i in range(len(recommend_song)):
      print(i+1)
      print(f"{recommend_song[i][1]} by {recommend_song[i][2]}")
      print(f"Similarity score : {round(recommend_song[i][0], 3)}")
    print(bow_songs)

In [40]:
def recommend_data(song,number_songs,matrix_similar):
    recommend_song_temp = matrix_similar[song][:(number_songs+1)]
    recommend_song = recommend_song_temp[1:number_songs+1]
    print_data(song, recommend_song)

In [41]:
selected_song = songs['song'].iloc[12]
number_song = input("Enter number of recommendations you want to receive : ")
number_song_int = int(number_song)

Enter number of recommendations you want to receive : 3


In [42]:
recommend_data(selected_song,number_song_int, similarities)

Recommended songs for Disillusion are: 
1
Rogues by Incubus
Similarity score : 0.176
2
Somebody Out There by Alan Parsons Project
Similarity score : 0.173
3
How Do I Stop Loving You by Engelbert Humperdinck
Similarity score : 0.132
        artist_data               song_data  ... zwei zydeco
0              ABBA   Ahe's My Kind Of Girl  ...  0.0    0.0
1              ABBA        Andante, Andante  ...  0.0    0.0
2              ABBA          As Good As New  ...  0.0    0.0
3              ABBA                    Bang  ...  0.0    0.0
4              ABBA        Bang-A-Boomerang  ...  0.0    0.0
...             ...                     ...  ...  ...    ...
9995  Justin Bieber                 All Bad  ...  0.0    0.0
9996  Justin Bieber       All I Want Is You  ...  0.0    0.0
9997  Justin Bieber  As Long As You Love Me  ...  0.0    0.0
9998  Justin Bieber               Baby Lady  ...  0.0    0.0
9999  Justin Bieber       Beauty And A Beat  ...  0.0    0.0

[10000 rows x 31429 columns]
